In [1]:
import json
from pathlib import Path
import subprocess


In [2]:
INPUT_FILE = Path("grouped_sessions.json")

with open(INPUT_FILE, "r", encoding="utf-8") as f:
    sessions = json.load(f)

print("Loaded sessions:", len(sessions))
sessions[:3]


Loaded sessions: 4


[{'start': '12:01',
  'end': '12:01',
  'task': 'User is tracking activity in a Python Jupyter notebook.'},
 {'start': '12:02',
  'end': '12:02',
  'task': 'The user is reviewing Tesseract OCR documentation.'},
 {'start': '12:04',
  'end': '12:04',
  'task': 'User is viewing a GitHub repository for an auto-timesheet system.'}]

In [3]:
timeline = ""

for s in sessions:
    timeline += f"{s['start']} - {s['end']} : {s['task']}\n"

print(timeline)


12:01 - 12:01 : User is tracking activity in a Python Jupyter notebook.
12:02 - 12:02 : The user is reviewing Tesseract OCR documentation.
12:04 - 12:04 : User is viewing a GitHub repository for an auto-timesheet system.
12:07 - 12:08 : The user is setting up Tesseract OCR software.



In [6]:
PROMPT = f"""
You are generating a professional end-of-day work summary.

Below is a timeline of REAL work sessions recorded during the day.

STRICT RULES:

- Use ONLY the information present in the timeline
- DO NOT invent activities
- DO NOT create zero-duration entries
- Merge nearby related sessions into broader work blocks
- Produce BETWEEN 2 AND 6 entries ONLY
- Each entry must span a meaningful duration (at least ~30 minutes logically)
- Keep time order correct
- If project name is relevant, include it naturally; otherwise DO NOT force it
- Write professional, realistic timesheet-style descriptions

Return ONLY valid JSON:

[
  {{
    "start": "HH:MM",
    "end": "HH:MM",
    "task": "Professional work description"
  }}
]

Timeline:
{timeline}
"""


In [8]:
import json
from pathlib import Path
import subprocess

result = subprocess.run(
    ["ollama", "run", "llama3.2:3b"],
    input=PROMPT.encode("utf-8"),
    capture_output=True
)

output = result.stdout.decode("utf-8", errors="ignore")
print(output)

# ---------- EXTRACT JSON ----------
start = output.find("[")
end = output.rfind("]") + 1

if start == -1 or end == -1:
    print("❌ JSON not found in LLM output")
else:
    json_text = output[start:end]

    try:
        data = json.loads(json_text)

        # ---------- SAVE JSON ----------
        Path("daily_summary.json").write_text(
            json.dumps(data, indent=2),
            encoding="utf-8"
        )

        print("✅ Saved clean JSON → daily_summary.json")

    except json.JSONDecodeError:
        print("❌ Invalid JSON from LLM")


[
  {
    "start": "12:01",
    "end": "12:02",
    "task": "Tracking activity in Python Jupyter notebook"
  },
  {
    "start": "12:04",
    "end": "12:08",
    "task": "Setting up Tesseract OCR software and reviewing documentation"
  }
]


✅ Saved clean JSON → daily_summary.json
